In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as urllib2
from io import StringIO
import datetime as dt
import hgutils
import time
hgt = hgutils.timer("AMFI Data Scraper")
import threading

In [2]:
def date_range(start_date,end_date,frequency=1):
    current_date = start_date
    yield current_date
    current_date = current_date + dt.timedelta(days=frequency)
    while current_date<=end_date:
        yield current_date
        current_date = current_date + dt.timedelta(days=frequency)

def generate_amfi_url (date,base_url = 'https://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?frmdt='):
    date_str = date.strftime("%d-%b-%Y")
    return base_url+date_str

def get_day_df (url, break_minutes = 5):
    page = urllib2.urlopen(url)
    soup = BeautifulSoup(page.read())
    df = pd.read_csv(StringIO(soup.get_text()),sep=';')
    df.dropna(subset='Net Asset Value',inplace=True)
    cols_to_keep = ['Scheme Code','Scheme Name',
        'Net Asset Value','ISIN Div Payout/ISIN Growth',
        'ISIN Div Reinvestment','Date']
    df = df[cols_to_keep]
    return df

def get_day_df_recursive (url, break_minutes = 5):
    try:
        page = urllib2.urlopen(url)
        soup = BeautifulSoup(page.read())
        df = pd.read_csv(StringIO(soup.get_text()),sep=';')
        df.dropna(subset='Net Asset Value',inplace=True)
    except:
        hgt.start("Break Time")
        time.sleep(break_minutes*60)
        df = get_day_df_recursive(url,break_minutes)
        hgt.stop()
    return df

In [3]:
saved_file_name = "amfi2.csv"
hgt.start('Daddy Timer')
start_date = dt.date(2018,1,1)
end_date = dt.date(2022,8,23)
combo_df = pd.DataFrame()
hgt.start("Year - " + start_date.strftime("%Y"))
for d in date_range(start_date,end_date):
    if (d.day == 1)&(d.month==1):
        combo_df.to_csv(saved_file_name)
        hgt.stop(print=True,verbose=0)
        hgt.start("Year - " + d.strftime("%Y"))
    hgt.start(d.strftime("%d-%b-%Y"))
    try:
        df = get_day_df(generate_amfi_url(d),break_minutes=5)
    except:
        combo_df.to_csv(saved_file_name)
        print("Break Time")
        hgt.start("Break Time")
        df = get_day_df_recursive(generate_amfi_url(d),break_minutes=5)
        hgt.stop(print=True,verbose=0)
    combo_df = pd.concat([combo_df,df])
    hgt.stop()
hgt.stop(print=True,verbose=1)

		 Year - 2018 - 7.7 milliseconds
		 Year - 2018 - 8.66 minutes
		 Year - 2019 - 11.88 minutes
		 Year - 2020 - 14.41 minutes
